In [783]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (11, 5)  #set default figure size
import numpy as np
from numpy import exp
from scipy.special import factorial
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import statsmodels.api as sm
from statsmodels.api import Poisson
from scipy import stats
from scipy.stats import norm
from scipy.stats import poisson
from statsmodels.iolib.summary2 import summary_col
from scipy.optimize import minimize
import mpmath
import math
from math import e
from math import gamma
import warnings
warnings.filterwarnings("ignore")
import scipy.stats as s

<b> Part 1

<b> 1. The Poisson Model
Consider the example related to billboard exposures from class. The associated data is in the file billboard.csv.
Write code to estimate the parameters of the Poisson model using maximum likelihood estimation (MLE).
Report your code, the estimated parameters and the maximum value of the log-likelihood.


In [265]:
df=pd.read_csv(r'C:\Users\cheta\Downloads\billboard.csv')
df

,EXPOSURES,PEOPLE
0,0,48
1,1,37
2,2,30
3,3,24
4,4,20
5,5,16
6,6,13
7,7,11
8,8,9
9,9,7


In [266]:
params=0.1

In [267]:
inputs=df.copy()
inputs

,EXPOSURES,PEOPLE
0,0,48
1,1,37
2,2,30
3,3,24
4,4,20
5,5,16
6,6,13
7,7,11
8,8,9
9,9,7


In [268]:
poisson_pmf = lambda k,l: k*np.log(l)-l*(np.log(e))-np.log(factorial(k))

In [269]:

def LL(params,inputs):
    lambda0=params
    e=inputs['EXPOSURES']
    p=inputs['PEOPLE']
    sum=0
    for i in range(len(inputs)):
        sum+=poisson_pmf(e[i],lambda0)*p[i]
    return sum

In [270]:
def NLL(params, inputs):
    return(-(LL(params, inputs)))

In [271]:
final=minimize(NLL,
         args=inputs,
         x0=params,
         bounds=[(0.000001, None)],
        tol=1e-10,
        options={'ftol' : 1e-8},)


In [272]:
final

      fun: array([929.04388273])
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([-2.27373677e-05])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 24
      nit: 10
     njev: 12
   status: 0
  success: True
        x: array([4.45599937])

In [273]:
print("The Value for Lambda is ",final.x[0],". The Value of Maximum Log likelihood is ",-1*final.fun[0] )

The Value for Lambda is  4.455999368640959 . The Value of Maximum Log likelihood is  -929.0438827273031


<b>2. The NBD Model
Next, write code (for the same dataset) to estimate the parameters of the NBD model using MLE. Report
your code, the estimated parameters and the maximum value of the log-likelihood. Evaluate the NBD model
vis-`a-vis the Poisson model; explain which is better and why.


In [340]:
params = np.array([0.1,0.1])

In [341]:
df=pd.read_csv(r'C:\Users\cheta\Downloads\billboard.csv')
df
inputs=df.copy()
inputs

,EXPOSURES,PEOPLE
0,0,48
1,1,37
2,2,30
3,3,24
4,4,20
5,5,16
6,6,13
7,7,11
8,8,9
9,9,7


In [342]:
pmf_nbd=lambda a,n,k,t: (np.log(gamma(n+k))-(np.log(gamma(n))+np.log(factorial(k))))+(n*(np.log(a)-np.log(a+t)))+(k*(np.log(t)-np.log(a+t)))

In [347]:
def NBDLL(params,inputs):
    e=inputs['EXPOSURES']
    p=inputs['PEOPLE']
    n=params[0]
    alpha=params[1]
    sum=0
    for i in range(len(inputs)):
        sum+=pmf_nbd(alpha,n,e[i],1)*p[i]
    return sum

In [348]:
def NLL(params, inputs):
    return(-(NBDLL(params, inputs)))

In [351]:
final=minimize(NLL,
         args=inputs,
         x0=params,
         bounds=[(0.000001, None),(0.000001, None)])


In [352]:
final

      fun: 649.6888274836756
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 6.82121023e-05, -3.41060513e-04])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 42
      nit: 13
     njev: 14
   status: 0
  success: True
        x: array([0.96925927, 0.21751771])

In [353]:
print("The Value for n is ",final.x[0]," and Alpha is ",final.x[1],".The Value of Maximum Log likelihood is ",-1*final.fun )

The Value for n is  0.9692592676813033  and Alpha is  0.21751770719953603 .The Value of Maximum Log likelihood is  -649.6888274836756


<b> 3. The Poisson Regression
Now consider the khakichinos.com example from class; The associated data is in the file khakichinos.csv.
Estimate all relevant parameters for Poisson regression using MLE. Report your code, the estimated parameters and the maximum value of the log-likelihood.

In [302]:
kc=pd.read_csv(r'C:\Users\cheta\Downloads\khakichinos.csv')
kc.drop(['ID'],axis=1,inplace=True)
inputs=kc.copy()

In [303]:
inputs

,NumberofVisits,LnInc,Sex,LnAge,HHSize
0,0,11.379394,1,3.871201,2
1,5,9.769956,1,4.043051,1
2,0,11.082143,0,3.332205,2
3,0,10.915088,1,3.951244,3
4,0,10.915088,1,2.833213,3
...,...,...,...,...,...
2723,0,9.528794,1,2.944439,2
2724,0,11.379394,0,3.970292,2
2725,0,11.191342,1,3.044522,3
2726,0,10.532096,1,2.890372,4


In [305]:
poisson_pmf = lambda k,l: k*np.log(l)-l*(np.log(e))-np.log(factorial(k))

In [306]:
params = np.array([0.01,0.01,0.1,0.01,0.01])

In [308]:
def LL(params,inputs):
    k=inputs['NumberofVisits'].tolist()
    kc1=inputs.drop(['NumberofVisits'],axis=1)
    a=kc1.to_numpy().tolist()
    beta=params
    lambda0=beta[4]
    beta=np.delete(beta, [4])
    lambdai=0
    sum2=0
    sum1=0
    
    for i in range(len(a)):
        sum1=0
        for j in range(len(a[i])):
            sum1=sum1+beta[j]*a[i][j]
        lambdai=lambda0*np.exp(sum1)
        sum2=sum2+poisson_pmf(k[i],lambdai)
    return sum2

In [309]:
def NegLL(params, inputs):
    return(-(LL(params, inputs)))

In [310]:
final=minimize(NegLL,
         args=inputs,
         x0=params,
         bounds=[(None, None), (None, None), (None, None), (None, None), (0.000001, None)])


In [311]:
print("The Value for lambda is ",final.x[4],"Beta parameters are ",final.x[0:4],".The Value of Maximum Log likelihood is ",-1*final.fun )

The Value for lambda is  0.043843803145908994 Beta parameters are  [ 0.09383211  0.00427661  0.58845273 -0.03590396] .The Value of Maximum Log likelihood is  -6291.49675885234


<b> 4. The NBD Regression
Consider the khakichinos.com example again. Estimate all relevant parameters for NBD Regression using
MLE. Report your code, the estimated parameters and the maximum value of the log-likelihood. Evaluate the
NBD regression vis-`a-vis the Poisson regression; explain which is better and why.

In [323]:
kc=pd.read_csv(r'C:\Users\cheta\Downloads\khakichinos.csv')
kc.drop(['ID'],axis=1,inplace=True)
inputs=kc.copy()

In [324]:
params = np.array([0.1,0.1,0.1,0.1,0.1,0.1])

In [330]:
pmf_nbd=lambda a,n,k,t: (np.log(gamma(n+k))-(np.log(gamma(n))+np.log(factorial(k))))+(n*(np.log(a)-np.log(a+t)))+(k*(np.log(t)-np.log(a+t)))


In [331]:
def NBDLL(params,inputs):
    k=inputs['NumberofVisits'].tolist()
    kc1=inputs.drop(['NumberofVisits'],axis=1)
    a=kc1.to_numpy().tolist()
    beta=params
    alpha=beta[5]
    n=beta[4]
    beta=np.delete(beta, [4])
    beta=np.delete(beta, [4])
    lambdai=0
    sum2=0
    sum1=0
    for i in range(len(a)):
        sum1=0
        for j in range(len(a[i])):
            sum1=sum1+beta[j]*a[i][j]
        t=math.exp(sum1)
        sum2=sum2+pmf_nbd(alpha,n,k[i],t)
    return sum2

In [332]:
def NegLL(params, inputs):
    return(-(NBDLL(params, inputs)))

In [333]:
final=minimize(NegLL,
         args=inputs,
         x0=params,
         bounds=[(None, None), (None, None), (None, None), (None, None), (0.000001, None),(0.000001, None)])


In [334]:
print("The Value for n is ",final.x[4]," and Alpha is ",final.x[5])
print("The values for the beta parameters are",final.x[0:4])
print("The Value of Maximum Log likelihood is ",-1*final.fun)

The Value for n is  0.13875675766248502  and Alpha is  8.197058003322073
The values for the beta parameters are [ 0.07347589 -0.00919222  0.90195373 -0.02435738]
The Value of Maximum Log likelihood is  -2888.96611657162


<b>Part - 2

<b>1. Read books.csv and generate two new datasets –
<br>(a) books01.csv, with the structure of the dataset used in the billboard exposures example (i.e., with
only two columns – (i) the number purchases, and (ii) the number of people making the corresponding
number of purchases), and

In [404]:
bk=pd.read_csv(r'C:\Users\cheta\Downloads\books.csv')

In [405]:
bk.loc[bk["domain"] == "amazon.com", "qty"] =0 

In [406]:
bk

,userid,education,region,hhsz,age,income,child,race,country,domain,date,product,qty,price
0,11443031,4.0,1.0,2,11.0,4,1,1,0,amazon.com,20070101,LEONARD WOOLF A BIOGRAPHY BOOKS VICTORIA GLEND...,0,19.80
1,11443031,4.0,1.0,2,11.0,4,1,1,0,amazon.com,20070101,THE VIEW FROM CASTLE ROCK STORIES BOOKS ALICE ...,0,17.13
2,11443031,4.0,1.0,2,11.0,4,1,1,0,amazon.com,20070101,WILLIAM JAMES IN THE MAELSTROM OF AMERICAN MOD...,0,19.80
3,11519009,NaN,2.0,3,5.0,3,1,2,0,amazon.com,20070101,DRIVEN,0,2.00
4,11519009,NaN,2.0,3,5.0,3,1,2,0,amazon.com,20070101,CHOCOLATE COVERED FORBIDDEN FRUIT,0,3.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40940,15557019,NaN,2.0,3,6.0,6,1,1,0,amazon.com,20071231,"YOUR HEALTH TODAY, BRIEF: CHOICES IN A CHANGIN...",0,49.38
40941,15557019,NaN,2.0,3,6.0,6,1,1,0,amazon.com,20071231,THE PIANO LESSON,0,3.50
40942,15557019,NaN,2.0,3,6.0,6,1,1,0,amazon.com,20071231,"THE ESSENTIAL BOGOSIAN: TALK RADIO, DRINKING I...",0,2.65
40943,15557019,NaN,2.0,3,6.0,6,1,1,0,amazon.com,20071231,CAT ON A HOT TIN ROOF,0,5.00


In [407]:
bk1=bk.groupby('userid', as_index=False).agg({'qty':'sum'})

In [408]:
bk1=bk1.groupby('qty',as_index=False).agg({'userid':'count'})

In [409]:
bk1.to_csv('Book1.csv')

In [410]:
bk1.columns=['Qty_Purchased','Number_of_People']

In [411]:
bk1

,Qty_Purchased,Number_of_People
0,0,7639
1,1,753
2,2,362
3,3,175
4,4,126
5,5,82
6,6,74
7,7,30
8,8,48
9,9,31


In [412]:
bk1.head()

,Qty_Purchased,Number_of_People
0,0,7639
1,1,753
2,2,362
3,3,175
4,4,126


In [413]:
bk1.tail()

,Qty_Purchased,Number_of_People
41,62,1
42,63,1
43,83,1
44,86,1
45,111,1


<b> (b) books02.csv, with the structure of the dataset used in the khakichinos.com example, with a new column
containing a count of the number of books purchased from barnesandnoble.com by each customer, while
keeping the demographic variables (remember to drop date, product, and price).

In [650]:
bk

,userid,education,region,hhsz,age,income,child,race,country,domain,date,product,qty,price,Common_flag
0,11443031,4.0,1.0,2,11.0,4,1,1,0,amazon.com,20070101,LEONARD WOOLF A BIOGRAPHY BOOKS VICTORIA GLEND...,0,19.80,1
1,11443031,4.0,1.0,2,11.0,4,1,1,0,amazon.com,20070101,THE VIEW FROM CASTLE ROCK STORIES BOOKS ALICE ...,0,17.13,1
2,11443031,4.0,1.0,2,11.0,4,1,1,0,amazon.com,20070101,WILLIAM JAMES IN THE MAELSTROM OF AMERICAN MOD...,0,19.80,1
3,11519009,NaN,2.0,3,5.0,3,1,2,0,amazon.com,20070101,DRIVEN,0,2.00,1
4,11519009,NaN,2.0,3,5.0,3,1,2,0,amazon.com,20070101,CHOCOLATE COVERED FORBIDDEN FRUIT,0,3.88,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40940,15557019,NaN,2.0,3,6.0,6,1,1,0,amazon.com,20071231,"YOUR HEALTH TODAY, BRIEF: CHOICES IN A CHANGIN...",0,49.38,1
40941,15557019,NaN,2.0,3,6.0,6,1,1,0,amazon.com,20071231,THE PIANO LESSON,0,3.50,1
40942,15557019,NaN,2.0,3,6.0,6,1,1,0,amazon.com,20071231,"THE ESSENTIAL BOGOSIAN: TALK RADIO, DRINKING I...",0,2.65,1
40943,15557019,NaN,2.0,3,6.0,6,1,1,0,amazon.com,20071231,CAT ON A HOT TIN ROOF,0,5.00,1


In [651]:
#Creating two separate tables for each domain. 
#This is done to make sure we dont miss the users from amazon who did not purchase from barnes and noble 
bk_barnes = bk.loc[bk['domain']== 'barnesandnoble.com']
bk_amazon=  bk.loc[bk['domain']== 'amazon.com']

In [652]:
#Removing product , date , price and domain columns from the tables
bk_barnes.drop(['product','date','price','domain'],axis=1,inplace=True)
bk_amazon.drop(['product','date','price','domain'],axis=1,inplace=True)

In [653]:
#Grouping by the indpendent variables and aggregating the qty
bk_barnes=bk_barnes.groupby(['userid','education','region','hhsz','age','income','child','race','country'],dropna=False,as_index=False).agg({'qty':'sum'})

In [654]:
#Grouping by the indpendent variables and aggregating the qty
bk_amazon=bk_amazon.groupby(['userid','education','region','hhsz','age','income','child','race','country'],dropna=False,as_index=False).agg({'qty':'sum'})

In [655]:
#creating a flag column with a dummy value. this is needed to find the users who bought from both the domains.
#Such users need to be counted in Barnes and noble table but not in amazon
bk['Common_flag']=0

In [656]:
# Adding separate flags for amazon and barnes and noble
bk.loc[bk["domain"] == "amazon.com", "Common_flag"] =1 
bk.loc[bk["domain"] == "barnesandnoble.com", "Common_flag"] =2 

In [657]:
# Aggregating the flag column using mean. 
# This is done to be able to identify common users between two domains.
# Users who only bought from amazon will have flag as 1 and users who only bought from barnesandnoble will have 2 as flag after taking the mean
# Users who have bought from both amazon and barnes and noble will have a mean of flag between 1 and 2 

bk_flag=bk.groupby(['userid'],as_index=False).agg({'Common_flag':'mean'})

In [658]:
#creating a separate table with users information where flag value lies between 1 and 2
# this results in a dataframe which has common users from both domains
diff=bk_flag.loc[(bk_flag['Common_flag'] > 1) & (bk_flag['Common_flag'] < 2)]
diff.drop(['Common_flag'],axis=1,inplace=True)
#adding another flag for ease of use.
diff['flag']='T'

In [659]:
diff

,userid,flag
0,6365661,T
18,9573834,T
19,9576277,T
21,9581009,T
35,9752844,T
...,...,...
9345,15676745,T
9364,15680846,T
9398,15688844,T
9427,15695968,T


In [660]:
#joining the common users from previous analysis with the Amazon users table using a left join
bk_amazon=pd.merge(bk_amazon,diff,how='left',on='userid')

In [661]:
bk_amazon

,userid,education,region,hhsz,age,income,child,race,country,qty,flag
0,6365661,5.0,1.0,2,11.0,7,0,1,0,0,T
1,6388054,2.0,4.0,1,6.0,5,0,1,0,0,NaN
2,6421559,5.0,4.0,4,5.0,6,0,1,0,0,NaN
3,6467806,NaN,2.0,2,6.0,3,0,1,0,0,NaN
4,6628110,4.0,4.0,5,4.0,7,1,1,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
8168,15701290,NaN,1.0,2,3.0,3,0,1,0,0,NaN
8169,15701618,NaN,2.0,3,7.0,2,0,1,0,0,NaN
8170,15701673,NaN,4.0,5,5.0,3,1,1,1,0,NaN
8171,15701704,NaN,3.0,3,5.0,7,1,1,0,0,NaN


In [662]:
# Removing all common users in amazon table using the flag
bk_amazon=bk_amazon.loc[bk_amazon["flag"] != "T"]

In [663]:
#doing a union between amazon and barnes and noble data set
bk2=pd.concat([bk_amazon,bk_barnes])
bk2.drop(['flag'],axis=1,inplace=True)

In [664]:
#Final table contains 9451 rows and 10 columns
bk2

,userid,education,region,hhsz,age,income,child,race,country,qty
1,6388054,2.0,4.0,1,6.0,5,0,1,0,0
2,6421559,5.0,4.0,4,5.0,6,0,1,0,0
3,6467806,NaN,2.0,2,6.0,3,0,1,0,0
4,6628110,4.0,4.0,5,4.0,7,1,1,0,0
5,6631403,5.0,3.0,1,10.0,3,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...
1807,15695968,1.0,2.0,5,10.0,2,1,1,0,5
1808,15696910,NaN,3.0,4,8.0,4,1,1,0,2
1809,15698055,NaN,3.0,4,4.0,4,1,1,0,9
1810,15698341,NaN,4.0,6,8.0,6,1,1,0,2


<b> Displaying the head and tail of the resulting dataset book2 after transformation.

In [634]:
bk2.head()

,userid,education,region,hhsz,age,income,child,race,country,qty
1,6388054,2.0,4.0,1,6.0,5,0,1,0,0
2,6421559,5.0,4.0,4,5.0,6,0,1,0,0
3,6467806,NaN,2.0,2,6.0,3,0,1,0,0
4,6628110,4.0,4.0,5,4.0,7,1,1,0,0
5,6631403,5.0,3.0,1,10.0,3,0,1,1,0


In [635]:
bk2.tail()

,userid,education,region,hhsz,age,income,child,race,country,qty
1807,15695968,1.0,2.0,5,10.0,2,1,1,0,5
1808,15696910,NaN,3.0,4,8.0,4,1,1,0,2
1809,15698055,NaN,3.0,4,4.0,4,1,1,0,9
1810,15698341,NaN,4.0,6,8.0,6,1,1,0,2
1811,15698605,NaN,4.0,1,11.0,2,0,1,0,1


<b>2. Develop a Poisson model using books01.csv. Report your code, the estimated parameters and the maximum
value of the log-likelihood (and any other information you believe is relevant).


In [779]:
inputs=bk1.copy()
inputs

,Qty_Purchased,Number_of_People
0,0,7639
1,1,753
2,2,362
3,3,175
4,4,126
5,5,82
6,6,74
7,7,30
8,8,48
9,9,31


In [637]:
poisson_pmf = lambda k,l: k*np.log(l)-l*(np.log(e))-np.log(factorial(k))

In [638]:
params=0.1

In [639]:
inputs.columns=['Qty_Purchased','Number_of_People']


In [640]:
def LL(params,inputs):
    lambda0=params
    e=inputs['Qty_Purchased']
    p=inputs['Number_of_People']
    sum=0
    for i in range(len(inputs)):
        sum+=poisson_pmf(e[i],lambda0)*p[i]
    return sum
    

In [641]:
def NLL(params, inputs):
    return(-(LL(params, inputs)))

In [642]:
def p_callback(params):
    print(params)

In [643]:
final=minimize(NLL,
         args=inputs,
         x0=params,
         bounds=[(0.000001, None)],callback=p_callback)


[1.1]
[0.86518938]
[0.69368856]
[0.75703656]
[0.74911788]
[0.74848505]
[0.7484922]


In [644]:
print("The estimated value for lambda is ",final.x) 
print("The value of Maximum Log likelihood is ",-1*final.fun)

The estimated value for lambda is  [0.7484922]
The value of Maximum Log likelihood is  [-18921.91842943]


<b>3. Develop a Poisson model using books02.csv, i.e., by ignoring the independent variables available. Report
your code, and confirm that the estimated parameters and the maximum value of the log-likelihood are identical
to those obtained with the Poisson model developed using books01.csv.

In [665]:
bk2.drop(['userid'],axis=1,inplace=True)

In [666]:
bk2

,education,region,hhsz,age,income,child,race,country,qty
1,2.0,4.0,1,6.0,5,0,1,0,0
2,5.0,4.0,4,5.0,6,0,1,0,0
3,NaN,2.0,2,6.0,3,0,1,0,0
4,4.0,4.0,5,4.0,7,1,1,0,0
5,5.0,3.0,1,10.0,3,0,1,1,0
...,...,...,...,...,...,...,...,...,...
1807,1.0,2.0,5,10.0,2,1,1,0,5
1808,NaN,3.0,4,8.0,4,1,1,0,2
1809,NaN,3.0,4,4.0,4,1,1,0,9
1810,NaN,4.0,6,8.0,6,1,1,0,2


In [667]:
inputs=bk2.copy()

In [696]:
params = 1

In [706]:
def LL(params,inputs):
    k=inputs['qty'].tolist()
    lambda0=params
    sum1=0
    for i in range(len(k)):
        sum1=sum1+(poisson_pmf(k[i],lambda0))
    return sum1

In [707]:
def NLL(params, inputs):
    return(-1*(LL(params, inputs)))

In [708]:
inputs[:3]

,education,region,hhsz,age,income,child,race,country,qty
1,2.0,4.0,1,6.0,5,0,1,0,0
2,5.0,4.0,4,5.0,6,0,1,0,0
3,NaN,2.0,2,6.0,3,0,1,0,0


In [709]:
LL(params,inputs)

-19249.61978239895

In [711]:
final=minimize(NLL,
         args=inputs,
         x0=params,
         bounds=[(0.000001, None)],callback=p_callback)

[0.66011651]
[0.77819378]
[0.75199981]
[0.74835226]
[0.7484935]
[0.7484913]


In [712]:
final

      fun: array([18921.91842944])
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.02073648])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 16
      nit: 6
     njev: 8
   status: 0
  success: True
        x: array([0.7484913])

In [715]:
print("The estimated value for lambda is ",final.x[0])
print("The Value of Maximum Log likelihood is ",-1*final.fun)
print("We get the same values as the previous question")

The estimated value for lambda is  0.7484913027336835
The Value of Maximum Log likelihood is  [-18921.91842944]
We get the same values as the previous question


<b>4. Develop an NBD model using books01.csv. Report your code, the estimated parameters and the maximum
value of the log-likelihood (and any other information you believe is relevant).


In [740]:
params = np.array([1,1])

In [741]:
pmf_nbd=lambda a,n,k,t: (np.log(gamma(n+k))-(np.log(gamma(n))+np.log(factorial(k))))+(n*(np.log(a)-np.log(a+t)))+(k*(np.log(t)-np.log(a+t)))


In [742]:
bk1

,Qty_Purchased,Number_of_People
0,0,7639
1,1,753
2,2,362
3,3,175
4,4,126
5,5,82
6,6,74
7,7,30
8,8,48
9,9,31


In [743]:
inputs=bk1.copy()

In [744]:
def NBDLL(params,inputs):
    e=inputs['Qty_Purchased']
    p=inputs['Number_of_People']
    n=params[0]
    alpha=params[1]
    sum=0
    for i in range(len(inputs)):
        sum+=(pmf_nbd(alpha,n,e[i],1))*p[i]
    return sum

In [745]:
def NLL(params, inputs):
    return(-(NBDLL(params, inputs)))

In [746]:
def p_callback(params):
    print(params)

In [747]:
final=minimize(NLL,
         args=inputs,
         x0=params,
         bounds=[(0.000001, None),(0.000001, None)],callback=p_callback)

[0.9996994  1.35725779]
[0.14907399 0.40179165]
[0.0979495 0.2639978]
[0.08217807 0.22148967]
[0.08139086 0.21936793]
[0.08134365 0.2192407 ]
[0.08134119 0.21923405]


In [748]:
print("The Value for n is ",final.x[0]," and Alpha is ",final.x[1])
print("The Value of Maximum Log likelihood is ",-1*final.fun)

The Value for n is  0.08134118622615552  and Alpha is  0.21923404867443047
The Value of Maximum Log likelihood is  -8603.49884703562


<b>5. Develop an NBD model using books02.csv (again, ignoring the variables available). Report your code, and
confirm that the estimated parameters and the maximum value of the log-likelihood are identical to those
obtained with the NBD model developed using books01.csv.
2 

In [721]:
inputs=bk2.copy()

In [722]:
params = np.array([1,1])

In [723]:
pmf_nbd=lambda a,n,k,t: (np.log(gamma(n+k))-(np.log(gamma(n))+np.log(factorial(k))))+(n*(np.log(a)-np.log(a+t)))+(k*(np.log(t)-np.log(a+t)))


In [735]:
def NBDLL(params,inputs):
    k=inputs['qty'].tolist()
    kc1=inputs.drop(['qty'],axis=1)
    beta=params
    alpha=beta[1]
    n=beta[0]
    sum1=0
    for i in range(len(k)):
        sum1=sum1+pmf_nbd(alpha,n,k[i],1)
    return sum1

In [736]:
def NegLL(params, inputs):
    return(-(NBDLL(params, inputs)))


In [737]:
final=minimize(NegLL,
         args=inputs,
         x0=params,
         bounds=[(0.000001, None),(0.000001, None)]
              ,callback=p_callback)

[0.99969942 1.35725546]
[0.14906563 0.401912  ]
[0.0979436  0.26407576]
[0.08216343 0.22152897]
[0.08137654 0.21940732]
[0.08132885 0.21927876]
[0.08132649 0.21927238]


In [738]:
final

      fun: 8603.726982003966
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-7891.64424038,  2926.90274518])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 63
      nit: 7
     njev: 21
   status: 0
  success: True
        x: array([0.08132649, 0.21927238])

In [749]:
print("The Value for n is ",final.x[0]," and Alpha is ",final.x[1])
print("The Value of Maximum Log likelihood is ",-1*final.fun)
print('The values are similar to the values obtained when using book1.csv')

The Value for n is  0.08134118622615552  and Alpha is  0.21923404867443047
The Value of Maximum Log likelihood is  -8603.49884703562
The values are similar to the values obtained when using book1.csv


<b> 6. Calculate the values of (i) reach, (ii) average frequency, and (iii) gross ratings points (GRPs) based on the
NBD model. Show your work.

reach for the NBD model is defined as 100*(1-P(x=0|n,alpha)), alpha value obtained after the MLE implementation is 0.2566150324283152 and n is  1.1846549361700596

In [750]:
params = np.array([1,1])
inputs=bk1.copy()
inputs.columns=['Qty_Purchased','Number_of_People']


In [751]:
pmf_nbd=lambda a,n,k,t: (np.log(gamma(n+k))-(np.log(gamma(n))+np.log(factorial(k))))+(n*(np.log(a)-np.log(a+t)))+(k*(np.log(t)-np.log(a+t)))


In [752]:
def NBDLL(params,inputs):
    e=inputs['Qty_Purchased']
    p=inputs['Number_of_People']
    n=params[0]
    alpha=params[1]
    sum=0
    for i in range(len(inputs)):
        sum+=pmf_nbd(alpha,n,e[i],1)*p[i]
    return sum


In [753]:
def NLL(params, inputs):
    return(-(NBDLL(params, inputs)))

def p_callback(params):
    print(params)


In [754]:
final=minimize(NLL,
         args=inputs,
         x0=params,
         bounds=[(0.000001, None),(0.000001, None)],callback=p_callback)

[0.9996994  1.35725779]
[0.14907399 0.40179165]
[0.0979495 0.2639978]
[0.08217807 0.22148967]
[0.08139086 0.21936793]
[0.08134365 0.2192407 ]
[0.08134119 0.21923405]


In [755]:
print("The Value for n is ",final.x[0]," and Alpha is ",final.x[1])
print("The Value of Maximum Log likelihood is ",-1*final.fun)

The Value for n is  0.08134118622615552  and Alpha is  0.21923404867443047
The Value of Maximum Log likelihood is  -8603.49884703562


In [757]:
n=final.x[0]
alpha=final.x[1]
pmf_0=(alpha/(alpha+1))**n
reach = 100*(1-pmf_0)
print('reach is',reach)

reach is 13.026639691492758


In [758]:
reach

13.026639691492758

In [759]:
#reach is 18.9%

In [760]:
E_x= n/alpha

In [761]:
print('E(x) is ',E_x )

E(x) is  0.3710244221551086


In [762]:
Average_Frequency=E_x/(1-pmf_0)

In [763]:
grps= reach*Average_Frequency

In [764]:
print('reach is ',reach)
print('Average Frequency is' ,Average_Frequency)
print('grps is ',grps)

reach is  13.026639691492758
Average Frequency is 2.8481974702763266
grps is  37.10244221551086


<b> 7. Identify all independent variables with missing values. How many values are missing in each? Drop any variable
with many missing values (specify how you are defining ‘many’). If the number of missing values are very few
(again, specify how you are defining ‘few’), delete the rows involved. For the remaining variables (if any),
replace the missing values with the means of the corresponding variables. Report your code.

In [569]:
bk2

,education,region,hhsz,age,income,child,race,country,qty
1,2.0,4.0,1,6.0,5,0,1,0,0
2,5.0,4.0,4,5.0,6,0,1,0,0
3,NaN,2.0,2,6.0,3,0,1,0,0
4,4.0,4.0,5,4.0,7,1,1,0,0
5,5.0,3.0,1,10.0,3,0,1,1,0
...,...,...,...,...,...,...,...,...,...
1807,1.0,2.0,5,10.0,2,1,1,0,5
1808,NaN,3.0,4,8.0,4,1,1,0,2
1809,NaN,3.0,4,4.0,4,1,1,0,9
1810,NaN,4.0,6,8.0,6,1,1,0,2


<b> Treating missing values 

In [570]:
#below are the number of missing values by each column in the books table.
print('Below is the list of columns and the number of nulls in each colum')
bk2.isna().sum()

Below is the list of columns and the number of nulls in each colum


education    6914
region         11
hhsz            0
age             1
income          0
child           0
race            0
country         0
qty             0
dtype: int64

In [584]:
books_2 = pd.DataFrame(bk2, columns = ["education", "region", "hhsz","age","income","child","race","country","qty"])
books_2.head()


,education,region,hhsz,age,income,child,race,country,qty
1,2.0,4.0,1,6.0,5,0,1,0,0
2,5.0,4.0,4,5.0,6,0,1,0,0
3,NaN,2.0,2,6.0,3,0,1,0,0
4,4.0,4.0,5,4.0,7,1,1,0,0
5,5.0,3.0,1,10.0,3,0,1,1,0


In [585]:
books_2.isna().sum()

education    6914
region         11
hhsz            0
age             1
income          0
child           0
race            0
country         0
qty             0
dtype: int64

In [586]:
#below is the summary of the education variable
books_2.education.describe()

count    2537.000000
mean        2.749310
std         1.430923
min         0.000000
25%         1.000000
50%         2.000000
75%         4.000000
max         5.000000
Name: education, dtype: float64

In [587]:
#imputing the mean of education in the education column instead of deleting it 
#since the proportion of missing values to the total rows is high
books_3=books_2.copy()
books_3.education.fillna(books_3.education.mean(), inplace = True)
books_3.head()


,education,region,hhsz,age,income,child,race,country,qty
1,2.00000,4.0,1,6.0,5,0,1,0,0
2,5.00000,4.0,4,5.0,6,0,1,0,0
3,2.74931,2.0,2,6.0,3,0,1,0,0
4,4.00000,4.0,5,4.0,7,1,1,0,0
5,5.00000,3.0,1,10.0,3,0,1,1,0


In [588]:
books_3.isna().sum()

education     0
region       11
hhsz          0
age           1
income        0
child         0
race          0
country       0
qty           0
dtype: int64

In [589]:
#deleting the nulls rows which have nulls in the region column since we have only 46 rows 
#which is negligible compared to the entire data set
books_3 = books_3.dropna(axis=0, subset=['region'])

In [590]:
books_3.isna().sum()

education    0
region       0
hhsz         0
age          1
income       0
child        0
race         0
country      0
qty          0
dtype: int64

In [591]:
#deleting the nulls rows which have nulls in the age column since we have only 46 rows 
#which is negligible compared to the entire data set
books_3 = books_3.dropna(axis=0, subset=['age'])

In [592]:
books_3.isna().sum()

education    0
region       0
hhsz         0
age          0
income       0
child        0
race         0
country      0
qty          0
dtype: int64

In [593]:
books_3

,education,region,hhsz,age,income,child,race,country,qty
1,2.00000,4.0,1,6.0,5,0,1,0,0
2,5.00000,4.0,4,5.0,6,0,1,0,0
3,2.74931,2.0,2,6.0,3,0,1,0,0
4,4.00000,4.0,5,4.0,7,1,1,0,0
5,5.00000,3.0,1,10.0,3,0,1,1,0
...,...,...,...,...,...,...,...,...,...
1807,1.00000,2.0,5,10.0,2,1,1,0,5
1808,2.74931,3.0,4,8.0,4,1,1,0,2
1809,2.74931,3.0,4,4.0,4,1,1,0,9
1810,2.74931,4.0,6,8.0,6,1,1,0,2


In [598]:
bk3=books_3

In [599]:
bk3

,education,region,hhsz,age,income,child,race,country,qty
1,2.00000,4.0,1,6.0,5,0,1,0,0
2,5.00000,4.0,4,5.0,6,0,1,0,0
3,2.74931,2.0,2,6.0,3,0,1,0,0
4,4.00000,4.0,5,4.0,7,1,1,0,0
5,5.00000,3.0,1,10.0,3,0,1,1,0
...,...,...,...,...,...,...,...,...,...
1807,1.00000,2.0,5,10.0,2,1,1,0,5
1808,2.74931,3.0,4,8.0,4,1,1,0,2
1809,2.74931,3.0,4,4.0,4,1,1,0,9
1810,2.74931,4.0,6,8.0,6,1,1,0,2


<b> 8. Incorporate the available customer characteristics and estimate all relevant parameters for Poisson regression
using MLE. Report your code, the estimated parameters and the maximum value of the log-likelihood (and any
other information you believe is relevant). What are the managerial takeaways — which customer characteristics
seem to be important?


In [765]:
inputs=bk3.copy()

In [766]:
bk3

,education,region,hhsz,age,income,child,race,country,qty
1,2.00000,4.0,1,6.0,5,0,1,0,0
2,5.00000,4.0,4,5.0,6,0,1,0,0
3,2.74931,2.0,2,6.0,3,0,1,0,0
4,4.00000,4.0,5,4.0,7,1,1,0,0
5,5.00000,3.0,1,10.0,3,0,1,1,0
...,...,...,...,...,...,...,...,...,...
1807,1.00000,2.0,5,10.0,2,1,1,0,5
1808,2.74931,3.0,4,8.0,4,1,1,0,2
1809,2.74931,3.0,4,4.0,4,1,1,0,9
1810,2.74931,4.0,6,8.0,6,1,1,0,2


In [767]:
params = np.array([1,1,1,1,1,1,1,1,1])

In [768]:
def LL(params,inputs):
    k=inputs['qty'].tolist()
    kc1=inputs.drop(['qty'],axis=1)
    a=kc1.to_numpy().tolist()
    beta=params
    lambda0=beta[8]
    beta=np.delete(beta, [8])
    lambdai=0
    sum2=0
    sum1=0
    for i in range(len(a)):
        sum1=0
        for j in range(len(a[i])):
            sum1=sum1+beta[j]*a[i][j]
        lambdai=lambda0*np.exp(sum1)
        sum2=sum2+poisson_pmf(k[i],lambdai)
    return sum2


In [769]:
def NLL(params, inputs):
    return(-1*(LL(params, inputs)))


In [770]:
final=minimize(NLL,
         args=inputs,
         x0=params,
         bounds=[(0.000001, None),(0.000001, None),(0.000001, None),(0.000001, None),(0.000001, None),(0.000001, None),(0.000001, None),(0.000001, None),(0.000001, None)],callback=p_callback)


[0.66666667 0.66666667 0.66666667 0.66666667 0.66666667 0.66666667
 0.66666667 0.66666667 0.66666667]
[0.65314381 0.65321818 0.65338741 0.65294012 0.65282299 0.65287473
 0.65316911 0.65312421 0.65163125]
[0.62711426 0.62738249 0.62791008 0.62656962 0.62616393 0.62627598
 0.6272005  0.62705196 0.62213998]
[0.60600077 0.60647869 0.60732939 0.60524451 0.60453898 0.60465152
 0.60614687 0.60589678 0.59764191]
[0.58291482 0.58367984 0.5849187  0.58201455 0.58090744 0.58095622
 0.58313458 0.58275443 0.57020062]
[0.56049853 0.56160369 0.56325652 0.55956759 0.55799027 0.55789686
 0.56079472 0.56026805 0.54282981]
[0.53772315 0.53923933 0.54135332 0.53689691 0.53475304 0.53441458
 0.53809739 0.5373999  0.51419944]
[0.51498783 0.5169833  0.51960256 0.51443213 0.51162432 0.51091691
 0.51543435 0.51454227 0.48468518]
[0.4921426  0.49469203 0.49786969 0.49206084 0.48847576 0.48724393
 0.492648   0.49153351 0.45395065]
[0.46925605 0.47243571 0.47623088 0.46989456 0.46540608 0.4634589
 0.46979588 0.46

[1.00000000e-06 1.00000000e-06 1.00000000e-06 2.82132300e-02
 1.82492694e-02 4.24980160e-02 1.00000000e-06 1.00000000e-06
 5.44561251e-01]
[1.00000000e-06 1.00000000e-06 1.00000000e-06 2.82647008e-02
 1.68242992e-02 4.94179108e-02 1.00000000e-06 1.00000000e-06
 5.45352117e-01]
[1.00000000e-06 1.00000000e-06 1.00000000e-06 2.68031636e-02
 1.57907725e-02 5.91698138e-02 1.00000000e-06 1.00000000e-06
 5.50013351e-01]
[1.00000000e-06 1.00000000e-06 1.00000000e-06 2.60713660e-02
 1.58603491e-02 6.05334214e-02 1.00000000e-06 1.00000000e-06
 5.52787997e-01]
[1.00000000e-06 1.00000000e-06 1.00000000e-06 2.59178037e-02
 1.59945173e-02 6.02361216e-02 1.00000000e-06 1.00000000e-06
 5.53071019e-01]
[1.00000000e-06 1.00000000e-06 1.00000000e-06 2.58954220e-02
 1.60173675e-02 6.00616017e-02 1.00000000e-06 1.00000000e-06
 5.53144174e-01]
[1.00000000e-06 1.00000000e-06 1.00000000e-06 2.58948978e-02
 1.60185434e-02 6.00522527e-02 1.00000000e-06 1.00000000e-06
 5.53149828e-01]


In [771]:
print("The Value for lambda is",final.x[8])
print("The values for the beta parameters are",final.x[0:8])
print("The Value of Maximum Log likelihood is ",-1*final.fun)

The Value for lambda is 0.553149828274452
The values for the beta parameters are [1.00000000e-06 1.00000000e-06 1.00000000e-06 2.58948978e-02
 1.60185434e-02 6.00522527e-02 1.00000000e-06 1.00000000e-06]
The Value of Maximum Log likelihood is  -18884.638464333206


<b>9. Estimate all relevant parameters for NBD regression using MLE. Report your code, the estimated parameters
and the maximum value of the log-likelihood (and any other information you believe is relevant). What are
the managerial takeaways — which customer characteristics seem to be important?

In [772]:
params = np.array([0.01,0.01,0.1,0.01,0.01,0.01,0.01,0.01,0.01,0.01])

In [773]:
pmf_nbd=lambda a,n,k,t: (np.log(gamma(n+k))-(np.log(gamma(n))+np.log(factorial(k))))+(n*(np.log(a)-np.log(a+t)))+(k*(np.log(t)-np.log(a+t)))


In [787]:
inputs=bk3.copy()
inputs

,education,region,hhsz,age,income,child,race,country,qty
1,2.00000,4.0,1,6.0,5,0,1,0,0
2,5.00000,4.0,4,5.0,6,0,1,0,0
3,2.74931,2.0,2,6.0,3,0,1,0,0
4,4.00000,4.0,5,4.0,7,1,1,0,0
5,5.00000,3.0,1,10.0,3,0,1,1,0
...,...,...,...,...,...,...,...,...,...
1807,1.00000,2.0,5,10.0,2,1,1,0,5
1808,2.74931,3.0,4,8.0,4,1,1,0,2
1809,2.74931,3.0,4,4.0,4,1,1,0,9
1810,2.74931,4.0,6,8.0,6,1,1,0,2


In [775]:
def NBDLL(params,inputs):
    k=inputs['qty'].tolist()
    kc1=inputs.drop(['qty'],axis=1)
    a=kc1.to_numpy().tolist()
    beta=params
    alpha=beta[9]
    n=beta[8]
    beta=np.delete(beta, [9])
    beta=np.delete(beta, [8])
    lambdai=0
    sum2=0
    sum1=0
    for i in range(len(a)):
        sum1=0
        for j in range(len(a[i])):
            sum1=sum1+beta[j]*a[i][j]
        t=math.exp(sum1)
        sum2=sum2+pmf_nbd(alpha,n,k[i],t)
    return sum2

In [776]:
def NegLL(params, inputs):
    return(-(NBDLL(params, inputs)))


In [777]:
final=minimize(NegLL,
         args=inputs,
         x0=params,
         bounds=[(None, None), (None, None), (None, None), (None, None),(None, None), (None, None), (None, None), (None, None),(0.000001, None),(0.000001, None)])


In [778]:
print("The Value for n is ",final.x[8]," and Alpha is ",final.x[9])
print("The values for the beta parameters are",final.x[0:8])
print("The Value of Maximum Log likelihood is ",-1*final.fun)

The Value for n is  0.09835769568046158  and Alpha is  0.0785426592415001
The values for the beta parameters are [-0.12936605 -0.09999939 -0.0030107   0.02986125  0.01733193  0.05690244
 -0.21812761 -0.06818342]
The Value of Maximum Log likelihood is  -8355.151876290514
